# Funktionssammlung zur Erzeugung von Trainingsdaten


### Trainingssätze aus Pumastammdaten

Schritt 1: Aufarbeitung der Stammdatentabelle:

In [1]:
def txt_to_list(filePath):
    with open(filePath, 'r', encoding='utf-8') as f:
        list = f.read().splitlines()
        list = [feld for feld in list if feld != '']
        list = sorted(set(list), key=list.index)
        list = sorted(list)
    return list

In [3]:
#################### Skript zum auslesen der PUMA-Stammdaten  ########################
### inputfile format: .xls, .xlsx
### ouptutfile format: .xlsx
### Erforderliche Dateien in Unterordner "Annotationslisten/" :
    # Position.txt : muss vollständige Liste der im inputfile verwendeten Positions-Bezeichnungen enthalten.
    # Baugruppen.txt : muss vollständige Liste der im inputfile verwendeten Baugruppen-Bezeichnungen enthalten.
    # Komponenten.txt : muss vollständige Liste der im inputfile verwendeten Komponenten-Bezeichnungen enthalten.
    # Zustand.txt : muss vollständige Liste der im inputfile verwendeten Schadbeschreibungen enthalten.
    
def txt_to_list(filePath):
    with open(filePath, 'r', encoding='utf-8') as f:
        list = f.read().splitlines()
        list = [feld for feld in list if feld != '']
        list = sorted(set(list), key=list.index)
        list = sorted(list)
    return list
    
def process_puma_stammdaten(inputfile, outputfile):
    # Imports:
    import pandas as pd
    
    #Variable
    zaehler=0
    vis='-----------'

    #Hinterlegte Listen:
    l_B_Position = txt_to_list("Annotationslisten/Position.txt")
    l_Baugruppe = txt_to_list("Annotationslisten/Baugruppe.txt")
    l_Komponente = txt_to_list("Annotationslisten/Komponente.txt")
    l_K_Zustand = txt_to_list("Annotationslisten/Zustand.txt")

    # import Excel in Dataframe:
    df = pd.read_excel(inputfile)
    
    # anlegen neuer Spalten
    df["Hauptabschnitt"] = ""
    df["Baugruppe"] = ""
    df["B_Position"] = ""
    df["B_Zustand"] = ""
    df["Komponente"] = ""
    df["K_Zustand"] = ""
    df["Hauptabschnitt"] = ""
    #Variable
    l_drop=[]
    last_Abschnitt = df.loc[0,"Hauptabschnitt"]
    last_Baugruppe = df.loc[0,"Baugruppe"]
    last_B_Position = df.loc[0,"B_Position"]

    #Iterating rows:
    for i in range(0, len(df.index)):
    #for i in range(0, 22):
    
        # Fortschrittanzeige
        if i > 0:
            if (100*i/len(df.index)) > 10*zaehler:
                print('Fortschritt: '+str(10*zaehler)+'/100 ['+vis+']')
                vis=vis[:zaehler]+'=>'+vis[zaehler+2:]
                zaehler+=1
        
        #Variable
        hauptabschnitt = ''
        baugruppe = ''
        b_position = ''
        l_Abschnitt = ['Fahrdrahtprüfung', 'Führerstand 1', 'Führerstand 2', 'Maschinenraum', 'Außen Führerstand 1', 'Außen links', 'Außen Führerstand 2', 'Außen rechts', 'Außen unterhalb', 'Dach', 'Ausrüstung']
        
        # Analysing column "Struktur-Beschreibung"
        if isinstance(df.loc[i,"Struktur-Beschreibung"], str):
            #extracting "Hauptabschnitt":
            for h in l_Abschnitt:
                if hauptabschnitt != '':
                    break
                if h == df.loc[i,"Struktur-Beschreibung"]:
                    hauptabschnitt = h
            df.loc[i, "Hauptabschnitt"] = hauptabschnitt
            #extract "Baugruppe":
            for j in l_Baugruppe:
                if j in df.loc[i,"Struktur-Beschreibung"]:
                    if baugruppe == '':
                        baugruppe = j
                    else:
                        baugruppe = baugruppe + '; ' + j
            df.loc[i, "Baugruppe"] = baugruppe
            if (df.loc[i, "Baugruppe"] == ''):########### .bool()
                df.loc[i, "Baugruppe"] = float("NaN")

            # extract Position of Baugruppe
            for l in l_B_Position:
                if l in df.loc[i,"Struktur-Beschreibung"]:
                    if b_position == '':
                        b_position = l
                    else:
                        b_position = b_position + '; ' + l
            df.loc[i, "B_Position"] = b_position  
            if (df.loc[i, "B_Position"] == ''): ######.bool()
                df.loc[i, "B_Position"] = float("NaN")

        # Analysing column Befundtext (Komponente, K_Position, K_Zustand)
        komponente = ''
        k_position = ''
        k_zustand = ''
        if isinstance(df.loc[i,"Befundtext"], str):

            # extract Komponente
            for j in l_Komponente:
                if j in df.loc[i,"Befundtext"]:
                    if komponente == '':
                        komponente = j
                    else:
                        komponente = komponente + '; ' + j
            df.loc[i, "Komponente"] = komponente
            if (df.loc[i, "Komponente"] == ''):####.bool()
                df.loc[i, "Komponente"] = float("NaN")


            # extract Zustand of Komponente
            for l in l_K_Zustand:
                if l in df.loc[i,"Befundtext"]:
                    if k_zustand == '':
                        k_zustand = l
                    else:
                        k_zustand = k_zustand + '; ' + l
            df.loc[i, "K_Zustand"] = k_zustand
            if (df.loc[i, "K_Zustand"] == ''):###.bool()
                df.loc[i, "K_Zustand"] = float("NaN")



        #assign Zustand either to Baugruppe or to Komponente
        if isinstance(df.loc[i,"K_Zustand"], str):
            #if no entrie for Komponente, Zustand will be assignt to Baugruppe
            if isinstance(df.loc[i,"Komponente"], float):
                df.loc[i,"B_Zustand"] = df.loc[i,"K_Zustand"]
                df.loc[i,"K_Zustand"] = float("NaN")
                
        #Nan Einträge und überflüssige Zeilen entfernen: 
        if isinstance(df.loc[i,"Hauptabschnitt"], str) and df.loc[i,"Hauptabschnitt"] != '':
            last_Abschnitt = df.loc[i,"Hauptabschnitt"]
        elif isinstance(df.loc[i,"Hauptabschnitt"], float):
            df.loc[i,"Hauptabschnitt"] = last_Abschnitt
        elif isinstance(df.loc[i,"Hauptabschnitt"], str) and df.loc[i,"Baugruppe"] == '':
            df.loc[i,"Hauptabschnitt"] = last_Abschnitt
            
        if isinstance(df.loc[i,"Baugruppe"], str) and df.loc[i,"Baugruppe"] != '':
            last_Baugruppe = df.loc[i,"Baugruppe"]
        elif isinstance(df.loc[i,"Baugruppe"], float):
            df.loc[i,"Baugruppe"] = last_Baugruppe
        elif isinstance(df.loc[i,"Baugruppe"], str) and df.loc[i,"Baugruppe"] == '':
            df.loc[i,"Baugruppe"] = last_Baugruppe

        if isinstance(df.loc[i,"B_Position"], str) and df.loc[i,"B_Position"] != '':
            last_B_Position = df.loc[i,"B_Position"]
        elif isinstance(df.loc[i,"B_Position"], float):
            df.loc[i,"B_Position"] = last_B_Position
        elif isinstance(df.loc[i,"B_Position"], str) and df.loc[i,"B_Position"] == '':
            df.loc[i,"B_Position"] = last_B_Position
        
        # delete rows where Zustand, Komponente und Position = nan or ''
        j=df.loc[i,"Komponente"]
        k=df.loc[i,"B_Zustand"]
        l=df.loc[i,"K_Zustand"]
        if (isinstance(j, float) or j == '') and (isinstance(k, float) or k=='') and (isinstance(l, float) or l ==''):
            l_drop.append(i)
        
        df.loc[i,"Baugruppe"] = df.loc[i,"Baugruppe"].replace(',','')

    df1=df        
    df1=df.drop(df.index[l_drop]) 
    df2 = df1.loc[:, ["Hauptabschnitt", "Baugruppe", "B_Position", "B_Zustand", "Komponente", "K_Zustand", "Arbeitscode", "Einbauort"]]
    print('Fortschritt: 100/100[==========>]')
    
    #export to excel:
    df2.to_excel(outputfile)
    return df2

In [4]:
process_puma_stammdaten(r"..\03 Trainingsdaten\01 Datengrundlagen\Puma_Stammdaten\E1462_gültigem Arbeitscode_Kopie.xls", 'Ergebnisdateien\Stammdaten_aufgearbeitet_test.xlsx')

Fortschritt: 0/100 [-----------]
Fortschritt: 10/100 [=>---------]
Fortschritt: 20/100 [==>--------]
Fortschritt: 30/100 [===>-------]
Fortschritt: 40/100 [====>------]
Fortschritt: 50/100 [=====>-----]
Fortschritt: 60/100 [======>----]
Fortschritt: 70/100 [=======>---]
Fortschritt: 80/100 [========>--]
Fortschritt: 90/100 [=========>-]
Fortschritt: 100/100[==========>]


,Hauptabschnitt,Baugruppe,B_Position,B_Zustand,Komponente,K_Zustand,Arbeitscode,Einbauort
4,Fahrdrahtprüfung,Stirnfenster,Führerstand 1; Führerstand 1 Fahrerseite,,Heizscheibe,ohne Funktion,EBCBCEF-BT,Fst. 1 FS
6,Fahrdrahtprüfung,Stirnfenster,Führerstand 1; Führerstand 1 Fahrerseite,Funktionsstörung,NaN,NaN,EBCBBFZ-IS,Fst. 1 FS
8,Fahrdrahtprüfung,Stirnfenster,Führerstand 1; Führerstand 1 Beimannseite,,Heizscheibe,ohne Funktion,EBCBCEF-BT,Fst. 1 BS
10,Fahrdrahtprüfung,Stirnfenster,Führerstand 1; Führerstand 1 Beimannseite,Funktionsstörung,NaN,NaN,EBCBBFZ-IS,Fst. 1 BS
13,Fahrdrahtprüfung,Nischenheizung,Führerstand 1,Funktionsstörung,NaN,NaN,EBLABFZ-IS,Fst. 1
...,...,...,...,...,...,...,...,...
2598,Ausrüstung,ZFM21,Führerstand 2,beschädigt; nicht instandsetzbar beschädigt,NaN,NaN,EBJDWLH-BT,.
2599,Ausrüstung,ZFM21,Führerstand 2,fehlt,NaN,NaN,EBGCZKK-BT,.
2600,Ausrüstung,ZFM21,Führerstand 2,wirkungslos,NaN,NaN,EBGCZKK-BT,.
2602,Ausrüstung,ZFM21,Führerstand 2,beschädigt; nicht instandsetzbar beschädigt,NaN,NaN,EBJDJZV-BT,.


Schritt 2: Expansion von Sätzen aus der Grammatik und den tabelarisch aufgearbeiteten Daten

In [13]:
def expand_Sentences(input_grammar, input_content):
    
    # Imports:
    import pandas as pd 
    import re

    # import content-Excel in Dataframe:
    df = pd.read_excel(input_content)
    
    # import Grammar as list:
    with open(input_grammar) as f:
        list = f.read().splitlines()
    
    #define variables:
    #sentences = []
    sentences = set()
    zaehler=0
    vis='-----------'

    #Iterating rows:
    for i in range(0, len(df.index)):
    #for i in range (0, 10):
        # Fortschrittanzeige
        if i > 0:
            if (100*i/len(df.index)) > 10*zaehler:
                print('Fortschritt: '+str(10*zaehler)+'/100 ['+vis+']')
                vis=vis[:zaehler]+'=>'+vis[zaehler+2:]
                zaehler+=1
        
        if isinstance((df.loc[i,"Hauptabschnitt"]), str):
            Abschnitt = df.loc[i,'Hauptabschnitt']
        else:
            Abschnitt = ''
            
        if isinstance(df.loc[i,'Baugruppe'], str):
            l_Baugruppe = df.loc[i,'Baugruppe'].split('; ')
            if len(l_Baugruppe)== 0:
                l_Baugruppe.append('')
        else:
            l_Baugruppe = ['']
            
        if isinstance(df.loc[i,'B_Position'], str):
            l_B_Position = df.loc[i,'B_Position'].split('; ')
            if len(l_B_Position)== 0:
                l_B_Position.append('')
        else:
            l_B_Position = ['']
            
        if isinstance(df.loc[i,'B_Zustand'], str):
            l_B_Zustand = df.loc[i,'B_Zustand'].split('; ')
            if len(l_B_Zustand)== 0:
                l_B_Zustand.append('')
        else: 
            l_B_Zustand = ['']
            
        if isinstance(df.loc[i,'Komponente'], str):
            l_Komponente = df.loc[i,'Komponente'].split('; ')
            if len(l_Komponente)== 0:
                l_Komponente.append('')
        else:
            l_Komponente = ['']
            
        if isinstance(df.loc[i,'K_Zustand'], str):
            l_K_Zustand = df.loc[i,'K_Zustand'].split('; ')
            if len(l_K_Zustand)== 0:
                l_K_Zustand.append('')
        else:
            l_K_Zustand = ['']

        # complete iteration:
        for Baugruppe in l_Baugruppe:
            for Komponente in l_Komponente:
                for Zustand_B in l_B_Zustand:
                    for Position in l_B_Position:
                        for Zustand_K in l_K_Zustand:
                            for ii in list:
                                ii = re.sub('{{',  '',    ii)  
                                ii = re.sub('}}',  '',    ii)  
                                if 'Komponente' in ii: 
                                    ii=ii.replace('Zustand', 'Zustand_K')
                                else:
                                    ii=ii.replace('Zustand', 'Zustand_B')
                                ii = ii.split(' ')
                                sentence = ''
                                for j in range(0,len(ii)):
                                    if ii[j] in ['Abschnitt', 'Baugruppe', 'Position', 'Komponente', 'Zustand_K', 'Zustand_B']:
                                        sentence = sentence +' '+ eval(ii[j])
                                    else:
                                        sentence = sentence +' '+ ii[j]
                                #sentences.append(sentence)
                                sentences.add(sentence)

    print('Fortschritt: 100/100[==========>]')
    sentences = [feld for feld in sentences if feld != '']
    """sentences = sorted(set(sentences), key=sentences.index)"""
    outputfile = 'Ergebnisdateien/expanded_'+input_grammar[:-4]+'.txt'
    with open(outputfile, 'w') as file: 
        for i in sentences:
            file.write(i + '\n')

In [26]:
expand_Sentences('Grammatik_keinund.txt', 'Ergebnisdateien\Stammdaten_aufgearbeitet.xlsx')

Fortschritt: 0/100 [-----------]
Fortschritt: 10/100 [=>---------]
Fortschritt: 20/100 [==>--------]
Fortschritt: 30/100 [===>-------]
Fortschritt: 40/100 [====>------]
Fortschritt: 50/100 [=====>-----]
Fortschritt: 60/100 [======>----]
Fortschritt: 70/100 [=======>---]
Fortschritt: 80/100 [========>--]
Fortschritt: 90/100 [=========>-]
Fortschritt: 100/100[==========>]


Schritt 3: Preprocessing der Expandierten Sätze:

In [28]:
def preprocess_exp(inputfile):
    l2=set()
    #open inputfile
    with open(inputfile, 'r') as f:
        l = f.read().splitlines()
    # abbreviation processing
    abk = {
    " <": " kleiner ",
    " >" : " größer ",
    " Schr.Kupl.": " Schrauben Kupplung ",
    " Nm": " Newtonmeter ",
    " mm": " Millimeter ",
    " mm": " Millimeter ",    
    " z.B.": " zum Beispiel ",
    " bzw.": " beziehungsweise ",
    " zul.": " zulässig ",
    " cm²": " Quadratzentimeter ",
    " ≥": " größer gleich ",
    " ≤": " kleiner gleich ",
    " mm/s": " Millimeter pro Sekunde ",
    " +/-": " Plus Minus ",
    " ±": " Plus Minus ",
    " z. B.:": " zum Beispiel ",
    " l ": " Liter ",
    "/": " oder ",
    " %": " Prozent ",
    " bzgl.": " bezüglich ",
    " Pa ": " Pascal ",
    " kN/FM ": " Kilonewton pro Fahrmotor ",
    "'0'": " Null ",
    " s ": " Sekunden ",
    " m ": " Meter ",
    " Ril": " Richtlinie ",
    "Nr.": " Nummer ",
    " V ": " Volt ",
    "° C": " Grad Celsius ",
    " Fa. ": " Firma ",
    " A ": " Ampere ",
    " kg": " Kilogramm ",
    " mOhm ": " Milli Ohm ",
    " kOhm ": " Kilo Ohm ",
    "STA": "Stromabnehmer",
    "FML": "Fahrmotorlüfter",
    "SR": "Stromrichter",
    "HBU": "Hilfsbetriebeumrichter",
    "Fst.": "Führerstand",
    "MR": "Maschinenraum",
    "HBG": "Hilfsbetriebegerüst",
    "NSG": "Niederspannungsgerüst",
    "HSG": "Hochspannungsgerüst",
    " li ": " links ",
    " re ": " rechts ",
    "FS": "Fahrerseite",
    "BS": "Beimannseite",
    "z. B.": "zum Beispiel",
    "+": "plus",
    " ggf ": " gegenenfalls "
    }
    for i in l:
        for j in abk.keys():
            if j in i:
                i=i.replace(j, abk[j])
        l2.add(i)
        
    #Save Data to Textfile
    outputfile = inputfile[:-4] + '_preprocessed.txt'
    with open(outputfile, 'w', encoding='utf-8') as file:
        file.write('\ufeff') #add bom
        for item in l2:
            file.write(item + '\n')
        

In [29]:
preprocess_exp('Ergebnisdateien\expanded_Grammatik_keinund.txt')

## Trainingssätze aus tabellarischen Instandhaltungshandbuch (DB-Regio)

### Erzeugung von Trainingsdatensatz

inpufile:
- Dateiformat: .xlsx / .xls
- Regelwerkstabelle von DB Regio
- Vorausgesetzt: Spalte 5 enthält die Regelwerksnamen
- Vorausgesetzt: Spalte 10 enthält die Texte der Arbeitsanweisungen
- Vorausgesetzt: Spalte 9 enthält Sicherheitshinweise
- Vorausgesetzt: Spalte 11 enthält Qualitätsanforderungen 
- Vorausgesetzt: Spalte 12 enthält zusätzliche Hinweise

outputfile:
- Dateiformat: .txt
- Dateiname: extracted_sentences+inputfile.txt

In [31]:
def extract_Sentences(inputfile):
    #Imports
    import pandas as pd

    #Read Excel in to Dataframe
    #df = pd.read_excel("HAS_901_12.xlsx")
    df = pd.read_excel(inputfile)

    # Filter Exceldata
    df_Titel = df['Column5'][5:].dropna()
    df_Arbeitstext = df['Column10'][5:].dropna()
    df_Stext = df['Column9'][5:].dropna()
    df_Q_Anf = df['Column11'][5:].dropna()
    df_Hinweise = df['Column12'][5:].dropna()

    #iterate into list
    l=[]
    for i in df_Titel:
        l.append(i)
    for i in df_Arbeitstext:
        l.append(i)
    for i in df_Stext:
        l.append(i)
    for i in df_Q_Anf:
        l.append(i)
    for i in df_Hinweise:
        l.append(i)

    #delete duplicate entries:
    l = sorted(set(l), key=l.index)

    #Save Data to Textfile
    outputfile = 'extracted_sentences_'+inputfile[:-5]+'.txt'
    with open(outputfile, 'w', encoding='utf-8') as file:
        for item in l:
            file.write(item + '\n')
    


In [ ]:
extract_Sentences(inputfile)

### Preprocessing von Datensätzen:

- Teilt Sätze in einer Zeile in meherere Zeile mit je einem Satz auf.
- Sortiert Zeilen aus in der bestimmte Frasen/Zeichenfolgen vorkommen. Relevante Frasen/Zeichenfolgen sind blacklist_frases definiert
- Abkürzungen, Symbole und Schreibweisen werden durch ausformulierte Form ersetzt.
- Klammerausdrücke werden ink. Inhalt entfernt
- Bestimmte Nummer / Seriennummern / Regelwerksnummer werden entfernt
- Doppeleinträge werden bereinigt


In [52]:
def preprocess(inputfile):
    #open inputfile
    with open(inputfile, encoding='utf-8') as f:
        l = f.read().splitlines()
     
    
    # split sentences befor "kein"
    l1 = []
    for i in l:
        if len(i) == 0:
            continue
        elif i[0] == ' ':
            i=i[1:]
        i = i.replace('keine', '\nkeine')
        i = i.replace('Keine', '\nKeine')
        i = i.replace('kein', '\nkein')
        i = i.replace('Kein', '\nKein')
        n = i.split('\n')
        for j in n:
            l1.append(j)
    # delete blacklist frases:
    blacklist_frases =['MATERIAL SAP Nr.']
    l2 = []
    for i in l1:
        test = True
        for j in blacklist_frases:
            if j in i:
                test = False
        if test:
            l2.append(i)
            
    #delete duplicate entries:
    l2 = sorted(set(l2), key=l2.index)
    l2 = sorted(l2)
    
    # abbreviation processing
    l3=[]
    abk = {
    " <": " kleiner ",
    " >" : " größer ",
    " Schr.Kupl.": " Schrauben Kupplung ",
    " Nm": " Newtonmeter ",
    " mm": " Millimeter ",
    " mm": " Millimeter ",    
    " z.B.": " zum Beispiel ",
    " bzw.": " beziehungsweise ",
    " zul.": " zulässig ",
    " cm²": " Quadratzentimeter ",
    " ≥": " größer gleich ",
    " ≤": " kleiner gleich ",
    " mm/s": " Millimeter pro Sekunde ",
    " +/-": " Plus Minus ",
    " ±": " Plus Minus ",
    " z. B.:": " zum Beispiel ",
    " l ": " Liter ",
    "/": " oder ",
    " %": " Prozent ",
    " bzgl.": " bezüglich ",
    " Pa ": " Pascal ",
    " kN/FM ": " Kilonewton pro Fahrmotor ",
    "'0'": " Null ",
    " s ": " Sekunden ",
    " m ": " Meter ",
    " Ril": " Richtlinie ",
    "Nr.": " Nummer ",
    " V ": " Volt ",
    "° C": " Grad Celsius ",
    "°C": " Grad Celsius ",
    " Fa. ": " Firma ",
    " A ": " Ampere ",
    " kg": " Kilogramm ",
    " mOhm ": " Milli Ohm ",
    "z. B.": "zum Beispiel",
    "\"+\"": "plus",
    "\"-\"": "minus",
    "kV": "kilovolt",
    "o.g.": "oben genannt",
    "mF": "mikrofarat",
    "„": '"',
    "“": '"',
    "Hz": "Hertz",
    "'51X01B'": "",
    " ggf ": " gegebenenfalls ",
    "ggf.": "gegebenenfalls"
    }
    for i in l2:
        for j in abk.keys():
            if j in i:
                i=i.replace(j, abk[j])
        l3.append(i)
    
    #remove brackets
    l4 = []
    import re
    for i in l3:
        i = re.sub("[\(\[].*?[\)\]]", "", i)
        i = re.sub("R \d\d\d\d.\d\d\d\d", "", i)
        i = re.sub("AA R \d\d\d\d.\d\d\d\d", "", i)
        i = re.sub("\dFle\d\d\d.\d.\d\d.\d\d\d.\d\d\d", "", i)
        i = re.sub("AHB \d\d\d\-\d\d\d", "", i)
        i = re.sub("Ril \d\d\d(.)\d\d\d\d(A-Z)\d\d", "", i)
        i = re.sub("Ril \d\d\d(.)\d\d\d\d", "", i)
        i = re.sub("\d\d\d(.)\d\d\d\d", "", i)
        i = re.sub("\d\d\d\d(.)\d\d\d", "", i)
        i = re.sub("\d\d\d(.)\d\d", "", i)
        i = re.sub("\d\d\d\d\d\d\d", "", i)
        i = re.sub("\d\d\d(.)\d\d\d\d[a-z]\d\d", "", i)
        l4.append(i)
    
    #Save Data to Textfile
    outputfile = inputfile[:-4] + '_preprocessed.txt'
    with open(outputfile, 'w', encoding = 'utf8') as file:
        file.write('\ufeff') #add bom
        for item in l4:
            file.write(item + '\n')
    return l4

In [53]:
preprocess('Datensatz_IH_Handbuch_preprocessed.txt')

['',
 'plus-Leitung und minus-Leitung mit Sechskantmutter und Scheibe an der Batterie positionieren und Sechskantschraube anziehen',
 '"Batteriehahn" ist ausgeschaltet ',
 '"Batteriehahn": Stellung prüfen und Warnschilder am Batteriehahn und Fremdeinspeisungsdose anbringen',
 '"Batteriehauptschalter" an der Rückwand im Führerraum 1 in Stellung "Aus" schalten',
 '"Batteriehauptschalter" an der Rückwand im Führerraum 1 in Stellung "Ein" schalten',
 '"Batteriehauptschalter": Stellung prüfen und Warnschilder am Batteriehauptschalter und Fremdeinspeisungsdose anbringen',
 '"Batterieschalter" an der Rückwand im Führerraum 1 in Stellung "Ein" schalten',
 '"Fahrtrichtungsschalter" im Führerraum 1 in Stellung "M" verlegen',
 '"Fahrtrichtungsschalter" im Führerraum 1 in Stellung "V" verlegen',
 '"Richtungsschalter" im Führerraum 1 in Stellung "0" verlegen',
 '"Richtungsschalter" im Führerraum 1 in Stellung "M" verlegen',
 '"Richtungsschalter" in Führerraum 1 in Stellung "V" verlegen',
 '"Stromab